In [1]:
import pandas as pd
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold, rdScaffoldNetwork
from rdkit.Chem import rdFMCS 
from rdkit import DataStructs, Chem
from scipy.spatial.distance import squareform, cdist, pdist
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles
import tmap
from faerun import Faerun
from tqdm import tqdm
from glob import glob
import os


import seaborn as sns

sns.set(style = 'white', font_scale=2)


In [2]:
def _calc_ecfp4(smiles):
    ecfp4 = AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles), radius = 2)    
    return ecfp4

def pairwise_dist_tanimoto(smiles_list):    
    MorganFP_list = [_calc_ecfp4(i) for i in smiles_list]
    TanimotoDist =[]   
    for i, fp1 in enumerate(MorganFP_list):
        for fp2 in MorganFP_list[i+1:]:
            s = DataStructs.TanimotoSimilarity(fp1,fp2)
            #Available similarity metrics include Tanimoto, Dice, 
            # Cosine, Sokal, Russel, Kulczynski, McConnaughey, and Tversky
            d = 1. - s #distance
            TanimotoDist.append(d)
    dist_matrix = squareform(TanimotoDist)

    return dist_matrix

def get_core_fw(smi):
    mol = Chem.MolFromSmiles(smi)
    core = MurckoScaffold.GetScaffoldForMol(mol)
    fw = MurckoScaffold.MakeScaffoldGeneric(core)
    return Chem.MolToSmiles(core), Chem.MolToSmiles(fw)

def get_mcs(mols):
    scd = rdFMCS.FindMCS(mols, completeRingsOnly=True,)
    sc = Chem.MolFromSmarts(scd.smartsString)
    ##get matched
    # mol = mols[0]
    # match_aidx = mol.GetSubstructMatches(sc)
    # frag = AllChem.MolFragmentToSmarts(mol, match_aidx[0])
    return sc


In [4]:
csvs = glob('../*.csv')
dim = 2048
data_save_folder = './info'

for csv in csvs:
    df = pd.read_csv(csv)


    df.index = df.index.astype(str)
    task_name = os.path.basename(csv).replace('.csv', '')
    smiles_list = df['smiles'].to_list()



    def rtuple(x):
        d = pd.Series(x).reset_index().set_index(0)['index'].to_dict()
        return [(k, v) for k, v in d.items()]

    md1 = {'Yes':1, 'No':0}
    md2 = {'train':1, 'test':0}
    pair_key1 = rtuple(md1)
    pair_key2 = rtuple(md2)

    df['subgroup1'] = df.cliff_mol
    df['subgroup2'] = df.split.map(md2)


    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    ECFP4_fps = [AllChem.GetMorganFingerprintAsBitVect(x,2,dim) for x in tqdm(mols, ascii=True)]
    ecfps = [tmap.VectorUchar(list(fp)) for fp in ECFP4_fps]
    enc = tmap.Minhash(dim,seed = 42)
    lf = tmap.LSHForest(dim)
    lf.batch_add(enc.batch_from_binary_array(ecfps))
    lf.index()

    cfg = tmap.LayoutConfiguration()
    cfg.k = 50
    cfg.kc = 50
    cfg.sl_scaling_min = 1.0
    cfg.sl_scaling_max = 1.0
    cfg.sl_repeats = 1
    cfg.sl_extra_scaling_steps = 2
    cfg.placer = tmap.Placer.Barycenter
    cfg.merger = tmap.Merger.LocalBiconnected
    cfg.merger_factor = 2.0
    cfg.merger_adjustment = 0
    cfg.fme_iterations = 800
    cfg.sl_scaling_type = tmap.ScalingType.RelativeToDesiredLength
    cfg.node_size = 1 / 2
    cfg.mmm_repeats = 1
    x, y, s, t, gp = tmap.layout_from_lsh_forest(lf, config = cfg)

    #======================================================

    c1 = df['y'].round(3)
    c2 = df['subgroup1']
    c3 = df['subgroup2']

    c = [c1, c2, c3] #具体的属性数据

    series_title = ['pActivity',  'cliff_mol',  'split'] ##具体的属性数据名称


    categorical = [False, True, True]

    cmap = ['jet_r', 'tab10', 'Set1']

    min_legend_label= [str(c1.min()), None,  None]
    max_legend_label= [str(c1.max()), None, None]

    labels = (df['smiles'] + "__" + df.index + "__" 
              + task_name + ': ' + df['exp_mean [nM]'].astype(str) + ' nM').tolist() #显示smiles， 以及图片中的标签

    point_scale = 10

    legend_labels = [None, pair_key1, pair_key2]
    #========================================================


    faerun = Faerun(view="front", clear_color='#111111',coords=False) #'#ffffff'
    faerun.add_scatter(task_name, { "x": x, "y": y, 
                                  "c": c, "labels": labels},
                       legend_labels = legend_labels,
                       categorical = categorical,
                       max_legend_label = max_legend_label,
                       min_legend_label = min_legend_label,
                       point_scale=point_scale,
                       colormap = cmap,
                       has_legend=True,
                       series_title = series_title,
                       shader = 'smoothCircle') #"sphere", #

    faerun.add_tree(task_name + "_tree", {"from": s, "to": t}, point_helper=task_name,  color='#666666', ) #colors when no value

    # Choose the "smiles" template to display structure on hover
    faerun.plot(task_name, path = data_save_folder, template="smiles", notebook_height=750)

100%|###########################################################################################################| 1721/1721 [00:00<00:00, 28470.78it/s]


/home/shenwanxiang/Research/MPCD/dataset/HSSMS/MoleculeACE_benchmark/space/info/CHEMBL239_EC50.html

100%|###########################################################################################################| 1865/1865 [00:00<00:00, 32555.53it/s]


/home/shenwanxiang/Research/MPCD/dataset/HSSMS/MoleculeACE_benchmark/space/info/CHEMBL219_Ki.html

100%|#############################################################################################################| 976/976 [00:00<00:00, 25542.15it/s]


...

/home/shenwanxiang/Research/MPCD/dataset/HSSMS/MoleculeACE_benchmark/space/info/CHEMBL4616_EC50.html

100%|#############################################################################################################| 794/794 [00:00<00:00, 33599.80it/s]


/home/shenwanxiang/Research/MPCD/dataset/HSSMS/MoleculeACE_benchmark/space/info/CHEMBL1862_Ki.html

100%|#############################################################################################################| 615/615 [00:00<00:00, 22082.65it/s]


/home/shenwanxiang/Research/MPCD/dataset/HSSMS/MoleculeACE_benchmark/space/info/CHEMBL2835_Ki.html

In [5]:
df

,smiles,exp_mean [nM],y,cliff_mol,split,subgroup1,subgroup2
0,C[C@@H]1CCN(C(=O)CC#N)C[C@@H]1N(C)c1ncnc2[nH]c...,0.70,0.154902,0,train,0,1
1,C[C@@H]1CCN(C(=O)CC#N)C[C@@H]1n1cnc2cnc3[nH]cc...,0.50,0.301030,0,test,0,0
2,C[C@@H]1CCN(Cc2ccccc2)C[C@@H]1N(C)c1ncnc2[nH]c...,610.00,-2.785330,0,test,0,0
3,C[C@@H]1CCN(Cc2ccccc2)C[C@@H]1n1cnc2cnc3[nH]cc...,12.00,-1.079181,0,train,0,1
4,N#CCC(=O)N1CCC[C@@H](n2cnc3cnc4[nH]ccc4c32)C1,0.40,0.397940,0,train,0,1
...,...,...,...,...,...,...,...
610,CC(C)(C#N)CNC1CCN(C(=O)Cn2cc(NC(=O)c3cnn4cccnc...,0.27,0.568636,0,train,0,1
611,CN(CC1(C#N)CC1)C1CCN(C(=O)Cn2cc(NC(=O)c3cnn4cc...,0.24,0.619789,0,train,0,1
612,N#CC1(CNC2CCN(C(=O)Cn3cc(NC(=O)c4cnn5cccnc45)c...,0.36,0.443697,0,test,0,0
613,Cc1cc(Nc2nccc3nc(-c4c(Cl)cc(C#N)cc4Cl)[nH]c23)...,24.00,-1.380211,0,train,0,1
